# using TensorFlow directly

In [6]:
import tensorflow as tf
import numpy as np

#import visualization
import PIL.image
from io import BytesIO
from IPython.display import Image, display

def DisplayFractal(a, fmt='jpeg'):
    """Display an array of iteration counts as a 
       colorful picture of a fractal"""
    a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
    img = np.concatenate([10+20*np.cos(a_cyclic),
                        30+50*np.sin(a_cyclic),
                        155-80*np.cos(a_cyclic)], 2)
    img[a==a.max()] = 0
    a = img
    a = np.uint8(np.clip(a, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
    # Use NumPy to create a 2D array of complex numbers

Y, X = np.mgrid[-1.3:1.3:0.005, -2:1:0.005]
Z = X+1j*Y

xs = tf.constant(Z.astype(np.complex64))
zs = tf.Variable(xs)
ns = tf.Variable(tf.zeros_like(xs, tf.float32))



# Operation to update the zs and the iteration count.
#
# Note: We keep computing zs after they diverge! This
#       is very wasteful! There are better, if a little
#       less simple, ways to do this.
#
for i in range(200):
    # Compute the new values of z: z^2 + x
    zs_ = zs*zs + xs

    # Have we diverged with this new value?
    not_diverged = tf.abs(zs_) < 4

    zs.assign(zs_),
    ns.assign_add(tf.cast(not_diverged, tf.float32))
    
DisplayFractal(ns.numpy())

ModuleNotFoundError: No module named 'PIL.image'

In [5]:
!conda install Pillow

Solving environment: ...working... done

# All requested packages already installed.



In [8]:
import tensorflow as tf

x=tf.Variable([1.0,2.0])
a=tf.constant([3.0,3.0])
sub = tf.subtract(x,a)
sub

<tf.Tensor: id=18, shape=(2,), dtype=float32, numpy=array([-2., -1.], dtype=float32)>

## Keras introduction which is higher lvl of Tensorflow

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']
#handle missing values of horsepower
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

#pandas to numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

#building the neural network
model = Sequential()#not reccurency/dense means every input connected to every neuron in next layer
model.add(Dense(25, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))#output
model.compile(loss='mean_squared_error',optimizer='adam')#for regression (1 x output) nn use mean_squared error
model.fit(x,y,verbose=2,epochs=100) #verbose means to print out data as it hoes
                                    #0-no data shown, 1-show progress

Train on 398 samples
Epoch 1/100
398/398 - 1s - loss: 965.3408
Epoch 2/100
398/398 - 0s - loss: 357.0622
Epoch 3/100
398/398 - 0s - loss: 268.7684
Epoch 4/100
398/398 - 0s - loss: 219.9442
Epoch 5/100
398/398 - 0s - loss: 203.4751
Epoch 6/100
398/398 - 0s - loss: 196.2473
Epoch 7/100
398/398 - 0s - loss: 189.3240
Epoch 8/100
398/398 - 0s - loss: 183.2852
Epoch 9/100
398/398 - 0s - loss: 177.8579
Epoch 10/100
398/398 - 0s - loss: 169.5128
Epoch 11/100
398/398 - 0s - loss: 164.0439
Epoch 12/100
398/398 - 0s - loss: 156.7169
Epoch 13/100
398/398 - 0s - loss: 149.5135
Epoch 14/100
398/398 - 0s - loss: 143.1391
Epoch 15/100
398/398 - 0s - loss: 136.1795
Epoch 16/100
398/398 - 0s - loss: 132.5765
Epoch 17/100
398/398 - 0s - loss: 126.9823
Epoch 18/100
398/398 - 0s - loss: 119.2639
Epoch 19/100
398/398 - 0s - loss: 113.8833
Epoch 20/100
398/398 - 0s - loss: 108.3287
Epoch 21/100
398/398 - 0s - loss: 107.1106
Epoch 22/100
398/398 - 0s - loss: 96.9317
Epoch 23/100
398/398 - 0s - loss: 87.9314
E

In [21]:
#getting output for input values based on our model
pred = model.predict(x)

In [22]:
#calcualte square error
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f'Final RMSE = {score}')

Final RMSE = 4.0804197157612485


In [26]:
for i in range(10):
    print(f'{i+1}.  {cars[i]} MPG: {y[i]} vs predicted {pred[i]}')

1.  chevrolet chevelle malibu MPG: 18.0 vs predicted [17.127596]
2.  buick skylark 320 MPG: 15.0 vs predicted [16.950739]
3.  plymouth satellite MPG: 18.0 vs predicted [18.000422]
4.  amc rebel sst MPG: 16.0 vs predicted [18.713633]
5.  ford torino MPG: 17.0 vs predicted [17.553774]
6.  ford galaxie 500 MPG: 15.0 vs predicted [13.0535145]
7.  chevrolet impala MPG: 14.0 vs predicted [13.289034]
8.  plymouth fury iii MPG: 14.0 vs predicted [13.367853]
9.  pontiac catalina MPG: 14.0 vs predicted [13.549585]
10.  amc ambassador dpl MPG: 15.0 vs predicted [15.52009]


# Example with flower classification


In [35]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/iris.csv", 
    na_values=['NA', '?'])

# convert to numpy
x = df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values
dummies = pd.get_dummies(df['species']) #classification
species = dummies.columns
y = dummies.values

#build neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) 
model.add(Dense(25,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')#for category classyfication
                                                                #best is categorical_crossentropy
model.fit(x,y,verbose=0, epochs=100)    

In [42]:
 pred = model.predict(x)

np.set_printoptions(suppress=True) #disable scientific notation
#print(pred)
predicted = np.argmax(pred,axis=1)
expected = np.argmax(y,axis=1)

In [43]:
print(f'Predictions = {predicted}')
print(f'expectictions = {expected}')

Predictions = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
expectictions = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [45]:
print(species[predicted[1:10]])

Index(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa'],
      dtype='object')


In [47]:
from sklearn.metrics import accuracy_score
correct = accuracy_score(expected, predicted)
correct

0.98

In [48]:
#predict some random data
sample_flower = np.array( [[5.0,3.0,4.0,2.0]], dtype=float)
pred = model.predict(sample_flower)
print(pred)
pred = np.argmax(pred)
print(f"Predict that {sample_flower} is: {species[pred]}")

[[0.00053413 0.23003595 0.7694299 ]]
Predict that [[5. 3. 4. 2.]] is: Iris-virginica


In [ ]:
sample_flower = np.array( [[5.0,3.0,4.0,2.0],[5.2,3.5,1.5,0.8]], dtype=float)
pred = model.predict(sample_flower)
print(pred)
pred = np.argmax(pred,axis=1)
print(f"Predict that these two flowers {sample_flower} are: {species[pred]}")


# Saving Neural Network

Complex neural networks will take a long time to fit/train. It is helpful to be able to save these neural networks so that they can be reloaded later. A reloaded neural network will not require retraining. Keras provides three formats for neural network saving.

YAML - Stores the neural network structure (no weights) in the YAML file format.
JSON - Stores the neural network structure (no weights) in the JSON file format.
HDF5 - Stores the complete neural network (with weights) in the HDF5 file format. Do not confuse HDF5 with HDFS. They are different. We do not use HDFS in this class.
Usually you will want to save in HDF5.

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
# . means current directiory
save_path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']
#handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

#pandas to numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Before save score (RMSE): {score}")

# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"), "w") as json_file:
    json_file.write(model_json)

# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(save_path,"network.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)
    
# !!!!!!!!!!!!!!! this method is saving weights as well !!!!!!!!!!!!!!!!!!
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"network.h5"))

Train on 398 samples
Epoch 1/100
398/398 - 5s - loss: 2101.4819
Epoch 2/100
398/398 - 0s - loss: 486.6413
Epoch 3/100
398/398 - 0s - loss: 249.8113
Epoch 4/100
398/398 - 0s - loss: 139.5084
Epoch 5/100
398/398 - 0s - loss: 123.5068
Epoch 6/100
398/398 - 0s - loss: 119.2850
Epoch 7/100
398/398 - 0s - loss: 117.8555
Epoch 8/100
398/398 - 0s - loss: 115.2269
Epoch 9/100
398/398 - 0s - loss: 115.5158
Epoch 10/100
398/398 - 0s - loss: 114.3842
Epoch 11/100
398/398 - 0s - loss: 111.7231
Epoch 12/100
398/398 - 0s - loss: 113.3247
Epoch 13/100
398/398 - 0s - loss: 106.5734
Epoch 14/100
398/398 - 0s - loss: 101.9074
Epoch 15/100
398/398 - 0s - loss: 110.6682
Epoch 16/100
398/398 - 0s - loss: 110.5674
Epoch 17/100
398/398 - 0s - loss: 104.0234
Epoch 18/100
398/398 - 0s - loss: 106.7666
Epoch 19/100
398/398 - 0s - loss: 94.7959
Epoch 20/100
398/398 - 0s - loss: 92.3798
Epoch 21/100
398/398 - 0s - loss: 92.5463
Epoch 22/100
398/398 - 0s - loss: 87.6070
Epoch 23/100
398/398 - 0s - loss: 85.5792
Epo

In [50]:
#loading model
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 3.798702782250574
